In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 73.1 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import sys

In [ ]:
target_list = ['FriendsAndFamily', 'Professionals', 'RespectiveIndividuals', 'Strangers']


In [ ]:
target_list

['FriendsAndFamily', 'Professionals', 'RespectiveIndividuals', 'Strangers']

In [ ]:
from transformers import BertTokenizer, BertModel

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
def get_input(title):
  inputs = tokenizer.encode_plus(
      title,
      None,
      add_special_tokens=True,
      max_length=256,
      padding='max_length',
      return_token_type_ids=True,
      truncation=True,
      return_attention_mask=True,
      return_tensors='pt'
  )
  return {
            'input_ids': inputs['input_ids'].flatten().reshape(1,256),
            'attention_mask': inputs['attention_mask'].flatten().reshape(1,256),
            'token_type_ids': inputs["token_type_ids"].flatten().reshape(1,256)
  }

In [ ]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.bert_model = BertModel.from_pretrained('bert-base-uncased', return_dict=True)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, 4)

    def forward(self, input_ids, attn_mask, token_type_ids):
        output = self.bert_model(
            input_ids,
            attention_mask=attn_mask,
            token_type_ids=token_type_ids
        )
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)
        return output

model = BERTClass()
device = torch.device('cpu')
model.load_state_dict(torch.load('/content/drive/MyDrive/Datasets/best_model.pth', map_location=device))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

In [ ]:
def test(text):
  model.eval()
  data = get_input(text)
  ids = data['input_ids'].to(device, dtype = torch.long)
  mask = data['attention_mask'].to(device, dtype = torch.long)
  token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
  with torch.no_grad():
    result = model(ids, mask, token_type_ids)
    result = result.reshape(4)
    result = torch.sigmoid(result)
    result = result.tolist()
    privacy_included = []
    for i in range(4):
      if result[i]>0.5:
        privacy_included.append(target_list[i])
    return privacy_included

In [ ]:
print(test("This is a sample text"))
print(test("I am leaving this country tomorrow"))
print(test("CEO of Google just introduced a large improvement in machine learning"))

['FriendsAndFamily', 'RespectiveIndividuals', 'Strangers']
['FriendsAndFamily']
['Professionals']


# Checking hidden outputs along with bias

In [ ]:
def test_hidden_layers(text):
  # Create an instance of the model
  model = BERTClass()

  model.eval()

  # Set the input tensors
  data = get_input(text)
  ids = data['input_ids'].to(device, dtype = torch.long)
  mask = data['attention_mask'].to(device, dtype = torch.long)
  token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)

  # Initialize lists to store hidden layer outputs, weights, and biases
  hidden_outputs = []
  hidden_weights = []
  hidden_biases = []
  with torch.no_grad():
    # Hook function to store hidden layer outputs
    def hook_fn(module, input, output):
        hidden_outputs.append(output)

    # Register the hook to the desired layers
    model.bert_model.register_forward_hook(hook_fn)
    model.linear.register_forward_hook(hook_fn)

    # Forward pass
    output = model(ids, mask, token_type_ids)

    # Access the weights and biases of the linear layer
    linear_weights = model.linear.weight
    linear_biases = model.linear.bias

    torch.set_printoptions(threshold=250)
    # Print the hidden layer outputs, weights, and biases
    print(f"Hidden Layer {0} Output:\n\n{hidden_outputs[0][1]}\n\nShape: {hidden_outputs[0][1].shape}")
    print(f"Hidden Layer {1} Output:\n\n{hidden_outputs[1]}\n\nShape: {hidden_outputs[1].shape}")
    print(f"Hidden Layer {1} Weight:\n\n{linear_weights}\n\nShape: {linear_weights.shape}")
    print(f"Hidden Layer {1} Bias:\n\n{linear_biases}\n\nShape: {linear_biases.shape}")
    print()

    print("Final Output:\n", output)

In [ ]:
test_hidden_layers("I am leaving this country tomorrow")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Hidden Layer 0 Output:

tensor([[-0.9070, -0.4910, -0.6997,  ..., -0.3091, -0.7173,  0.9310]])

Shape: torch.Size([1, 768])
Hidden Layer 1 Output:

tensor([[-0.2188,  0.2647, -0.0303,  0.1550]])

Shape: torch.Size([1, 4])
Hidden Layer 1 Weight:

Parameter containing:
tensor([[ 0.0148, -0.0134, -0.0104,  ..., -0.0001, -0.0061, -0.0330],
        [-0.0359,  0.0277,  0.0017,  ...,  0.0110, -0.0289, -0.0214],
        [ 0.0341, -0.0055, -0.0206,  ..., -0.0107, -0.0264, -0.0027],
        [-0.0203, -0.0062, -0.0255,  ..., -0.0056, -0.0304,  0.0276]],
       requires_grad=True)

Shape: torch.Size([4, 768])
Hidden Layer 1 Bias:

Parameter containing:
tensor([-0.0121,  0.0102,  0.0040,  0.0173], requires_grad=True)

Shape: torch.Size([4])

Final Output:
 tensor([[-0.2188,  0.2647, -0.0303,  0.1550]])
